**Problem:**

You are given the following dataset:
1. **Audible Data** : https://1drv.ms/u/s!AiqdXCxPTydhoog8ckLN-6Cw55fzIg?e=EWgZ5d

Your task is to:
- Find the problems with the datasets.
- Define the Data Quality Dimensions.
- Try to clean the datasets.

In [1]:
import pandas as pd
import numpy as np

In [24]:
data = pd.read_excel("./audible_uncleaned.xlsx")
data.head()

,name,author,narrator,time,releasedate,language,stars,price
0,Geronimo Stilton #11 & #12,Writtenby:GeronimoStilton,Narratedby:BillLobely,2 hrs and 20 mins,2008-04-08 00:00:00,English,5 out of 5 stars34 ratings,468
1,The Burning Maze,Writtenby:RickRiordan,Narratedby:RobbieDaymond,13 hrs and 8 mins,2018-01-05 00:00:00,English,4.5 out of 5 stars41 ratings,820
2,The Deep End,Writtenby:JeffKinney,Narratedby:DanRussell,2 hrs and 3 mins,2020-06-11 00:00:00,English,4.5 out of 5 stars38 ratings,410
3,Daughter of the Deep,Writtenby:RickRiordan,Narratedby:SoneelaNankani,11 hrs and 16 mins,2021-05-10 00:00:00,English,4.5 out of 5 stars12 ratings,615
4,"The Lightning Thief: Percy Jackson, Book 1",Writtenby:RickRiordan,Narratedby:JesseBernstein,10 hrs,2010-01-13 00:00:00,English,4.5 out of 5 stars181 ratings,820


### 1. author

In [26]:
data['author'] = data['author'].str.split(":").str.get(1)

In [29]:
def split_name(text):
    for i, char in enumerate(text):
        if char.isupper() and i>0:
            return text[:i] + " " + text[i:]

In [30]:
text = "RickRiordon"
split_name(text)

'Rick Riordon'

In [32]:
data['author'] = data['author'].apply(split_name)

In [33]:
data['author'][:10]

0    Geronimo Stilton
1        Rick Riordan
2         Jeff Kinney
3        Rick Riordan
4        Rick Riordan
5     Suzanne Collins
6       Winter Morgan
7        Rick Riordan
8    Mary PopeOsborne
9        Rick Riordan
Name: author, dtype: object

In [37]:
idx = data[data['author'].fillna("missing").str.contains("Ð")].index
data.drop(index=idx, inplace=True)

In [38]:
for i in data['author'].unique():
    print(i)

Geronimo Stilton
Rick Riordan
Jeff Kinney
Suzanne Collins
Winter Morgan
Mary PopeOsborne
Philip Pullman
Shannon Messenger
Christopher Paolini
Eoin Colfer
A. A.Milne
Dan Gutman
Tui T.Sutherland
Nayomi Phillips
Ruskin Bond
Carolyn Keene
Franklin W.Dixon
Michael Bond
Julia Donaldson,AxelScheffler
Brandon Mull
Dav Pilkey
Dusti Bowling
Julie Sykes
Stella Robson
Jude Watson
Francisco DÃ­azValladares
None
Vegetta777, Willyrex
Roshani Chokshi
Sophie Schoenwald,NadineReitz-Illustrator
Holly Rivers
Leah Cypess
Bertrand Fichou,NoraThullin,CatherinedeLasa,
Orianne Lallemand
Adam Blade
Jessica Khoury
Yoon HaLee
Jonathan Stroud
Mark Cheverton
Jessica Renwick
Neil Gaiman
Lee Bacon
R. L.Stine
David Walliams
Devin Hunter
Lori M.Lee
M. G.Leonard,SamSedgman
Matt Haig
Peter Lerangis
Robin Stevens
M. G.Leonard,SamSedgman,ElisaPaganelli-illustrator
Tom Fletcher
Terri Farley
Shannon Hale,DeanHale
Tracey West
Mattel, MartaCisaMuÃ±oz-traductor
C. J.Redwine
Katherine Applegate
Gertrude ChandlerWarner
C. S.Lewis

In [234]:
# some values consist only of some special characters like the one shown below
data['author'][87192]

'æ£®æœ¬å“²éƒ Ž'

In [237]:
data['author'] = data['author'].str.encode("ascii", "ignore").str.decode("ascii")

In [239]:
data['author'][87192]

' '

## 2.stars

In [50]:
data['rating'] = data['stars'].str.split(" ").str.get(0)
idx= data[data['rating'] == 'Not'].index.tolist()  # not rated yet
data.loc[idx, 'rating'] = np.nan

In [53]:
data['rating'] = pd.to_numeric(data['rating'])

In [54]:
text = "128 ratings"
import re

pattern = r"\d+ ratings"
re.findall(pattern, text)

['128 ratings']

In [67]:
# extracting number of ratings
def extract_ratings(text):
    pattern = "\d+ ratings"
    result = re.findall(pattern, text)
    return result

In [86]:
data['num_ratings'] = data['stars'].apply(extract_ratings).str.get(0).str.split(" ").str.get(0)

In [92]:
data['num_ratings'] = pd.to_numeric(data['num_ratings'])

## time

In [125]:
data['time'].unique()

array(['2 hrs and 20 mins', '13 hrs and 8 mins', '2 hrs and 3 mins', ...,
       '40 hrs and 58 mins', '24 hrs and 8 mins', '24 hrs and 4 mins'],
      dtype=object)

In [182]:
idx = data[data['time'].fillna("missing").str.contains("Less")].index  # in these rows, time="Less than 1 minute"
data

,name,author,narrator,time,releasedate,language,stars,price,rating,num_ratings
1401,The Story of Ice Cream,Stacy Taus-Bolstad,Narratedby:BookBuddyDigitalMedia,Less than 1 minute,2021-01-01 00:00:00,English,Not rated yet,164,NaN,NaN
1403,The Story of Salt,Lisa Owings,Narratedby:BookBuddyDigitalMedia,Less than 1 minute,2021-01-01 00:00:00,English,Not rated yet,164,NaN,NaN
1404,The Story of Milk,Stacy Taus-Bolstad,Narratedby:BookBuddyDigitalMedia,Less than 1 minute,2021-01-01 00:00:00,English,Not rated yet,164,NaN,NaN
1408,The Story of an Apple,Stacy Taus-Bolstad,Narratedby:BookBuddyDigitalMedia,Less than 1 minute,2021-01-01 00:00:00,English,Not rated yet,164,NaN,NaN
1409,We Like the Summer,Katie Peters,Narratedby:BookBuddyDigitalMedia,Less than 1 minute,2021-01-01 00:00:00,English,Not rated yet,164,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
87171,ç¬¬äºŒåäº”è©±ã‚µãƒ³ãƒ»ãƒŸã‚·ã‚§ãƒ«ã®ã„ã„ã...,æ£®æœ¬å“²éƒ Ž,Narratedby:å°é‡Žç”°è‹±ä¸€,Less than 1 minute,20-11-15,japanese,Not rated yet,139,NaN,NaN
87175,ç¬¬ä¹è©±ã‚ªãƒ©ãƒ³æœ€å¾Œã®å¤•ã¹ï¼šã¼ãã®æ...,æ£®æœ¬å“²éƒ Ž,Narratedby:å°é‡Žç”°è‹±ä¸€,Less than 1 minute,19-11-15,japanese,Not rated yet,139,NaN,NaN
87176,ç¬¬ä¸€è©±ãƒªãƒ¥ãƒ¼ãƒ™ãƒƒã‚¯ã®è¿½æ†¶:ã¼ãã®...,æ£®æœ¬å“²éƒ Ž,Narratedby:å°é‡Žç”°è‹±ä¸€,Less than 1 minute,23-07-15,japanese,Not rated yet,139,NaN,NaN
87180,ç¬¬ä¸ƒè©±ãƒã‚°ãƒ€ãƒ¼ãƒ‰ã®èŒ¶å±‹:ã¼ãã®æ—…...,æ£®æœ¬å“²éƒ Ž,Narratedby:å°é‡Žç”°è‹±ä¸€,Less than 1 minute,13-07-15,japanese,Not rated yet,139,NaN,NaN


In [184]:
data['hrs'] = data['time'].str.split("and").str.get(0).str.strip().str.split(" ").str.get(0)
data['mins'] = data['time'].str.split("and").str.get(1).str.strip().str.split(" ").str.get(0)

In [187]:
idx = data[data['hrs'].str.contains("Less")].index # rows where time="Less than 1 minute"
data.loc[idx, 'hrs'] = 0

In [188]:
data['hrs'] = pd.to_numeric(data['hrs'])
data['mins'] = pd.to_numeric(data['mins'])

In [190]:
data['time_fixed'] = data['hrs'] * 60 + data['mins']

In [194]:
data.drop(columns=['hrs', 'mins'], inplace=True)

## language

In [193]:
data['language'].unique()

array(['English', 'Hindi', 'spanish', 'german', 'french', 'catalan',
       'swedish', 'italian', 'danish', 'finnish', 'dutch', 'hebrew',
       'polish', 'galician', 'afrikaans', 'icelandic', 'romanian',
       'japanese', 'tamil', 'portuguese', 'russian', 'urdu', 'hungarian',
       'czech', 'mandarin_chinese', 'basque', 'korean', 'arabic',
       'bulgarian', 'greek', 'turkish', 'ukrainian', 'slovene',
       'norwegian', 'telugu', 'lithuanian'], dtype=object)

In [208]:
data['language'] = data['language'].str.lower().str.strip()

## releasedate

In [199]:
data['releasedate'] = pd.to_datetime(data['releasedate'])

## price

In [211]:
data['price'].unique()

array([468, 820, 410, ..., 810, 710, 544], dtype=object)

In [213]:
idx = data[data['price'] == "Free"].index
data.loc[idx, "price"] = 0

In [214]:
data['price'] = pd.to_numeric(data['price'])

In [215]:
data.columns

Index(['name', 'author', 'narrator', 'time', 'releasedate', 'language',
       'stars', 'price', 'rating', 'num_ratings', 'time_fixed'],
      dtype='object')

In [216]:
data.drop(columns=['time', 'stars'], inplace=True)

## narrator
characters = ©, Ã, â, €, ™, , Å

In [220]:
data['narrator'] = data['narrator'].str.split(":").str.get(1)

In [223]:
data['narrator'] = data['narrator'].apply(split_name)

In [233]:
data['narrator'] = data['narrator'].str.replace("©", "")
data['narrator'] = data['narrator'].str.replace("Ã", "")
data['narrator'] = data['narrator'].str.replace("â", "")
data['narrator'] = data['narrator'].str.replace("€", "")
data['narrator'] = data['narrator'].str.replace("™", "")
data['narrator'] = data['narrator'].str.replace("","")
data['narrator'] = data['narrator'].str.replace("Å","")
data['narrator'] = data['narrator'].str.replace(",","")

In [228]:
data['narrator'] = data['narrator'].str.split(",").str.get(0)

In [241]:
data['narrator'] = data['narrator'].str.encode("ascii", "ignore").str.decode("ascii")

In [232]:
for i in data['narrator'].unique():
    print(i)

Bill Lobely
Robbie Daymond
Dan Russell
Soneela Nankani
Jesse Bernstein
Tatiana Maslany
Luke Daniels
Mary PopeOsborne
Michael Crouch
Philip Pullman
Bill Lobley
Caitlin Kelly
Gerrard Doyle
Gerry O'Brien
Peter Dennis
Jane Collingwood
Michael Goldstrom
Shannon McManus
Monica Rachelle
Neha Gargava
Jorjeana Marie
Christopher Gebauer
Tim Gregory
Walter Lewis
Stephen Fry
Paul Boehmer
Imelda Staunton
E. B.Stevens
Len Forgione
Stacy Gonzalez
Kristin Atherton
Derek Steel
David Pittu
Nuria Sams³
Vanessa PrezJurado
lexde Porrata
Bernd Reheuser
Susie Trayling
Jessica Almasy
Guy Chappelier
Will Production
Theo Solomon
Michael Gallagher
Keylor Leigh
Elissa Park
Steven Pacey
Summer McCusker
Nick Walther
Reba Buhr
Neil Gaiman
Paul Panting
Jordan Fisher
David Walliams
Adam Verner
Ramon DeOcampo
Samantha Tan
Elisa Paganelli
Katie Leung
Jot Davies
Tom Felton
Natalie Budig
Julia Whelan
Emily Ellet
Miriam Monlleo
Mia SinclairJenness
Aimee Lilly
Kenneth Branagh
Nicole Davis
Adnan Kapadia
Oliver Chris
Malin Ly

## name

In [242]:
data['name'] = data['name'].str.encode("ascii", "ignore").str.decode("ascii")

In [243]:
data.sample(2)

,name,author,narrator,releasedate,language,price,rating,num_ratings,time_fixed
68333,Hope Diamond,i Minds,Ellouise Rothwell,2009-09-17,english,65.0,NaN,NaN,NaN
62135,It's Your Loss,"D K,EmmaHopkinson,RobynDonaldson",Robyn Donaldson,2021-09-23,english,888.0,NaN,NaN,409.0


In [244]:
data.rename(columns={'time_fixed':'time'}, inplace=True)